In [1]:
import pyspark

sc = pyspark.SparkContext(appName="DATA228_Project")



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/16 19:39:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Importing all necessary libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('DATA228_Project').getOrCreate()

In [3]:
from pyspark.ml.feature import StopWordsRemover, CountVectorizer,Tokenizer, StringIndexer, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes,LogisticRegression 
from pyspark.sql.functions import *

## Loading data into pyspark data frame

In [5]:
#import sf_data into pyspark data frame

df = spark.read.format('csv').option('header','true')\
.option('inferSchema', 'true').option('timestamp', 'true').load('SF_data.csv')


## Total records

In [6]:
df.count()

2129525

## Data Cleaning and Preprocessing

In [8]:
df1=df.select(str.strip('PdId'),'Incident Code','Category','Descript','DayOfWeek','Date','Time','PdDistrict','X','Y')
#display(df1)

In [9]:
df22=df1.dropna()

In [10]:
df2=df22.filter(col('Category')!=col('Descript'))

## EDA

### Number of incidents in each category

In [11]:
df4=df2.groupBy('Category').count()
(df4.show())

+--------------------+------+
|            Category| count|
+--------------------+------+
|               FRAUD| 41348|
|             SUICIDE|  1230|
|         LIQUOR LAWS|  2840|
|     SECONDARY CODES| 22378|
|      MISSING PERSON| 44268|
|      OTHER OFFENSES|301874|
|DRIVING UNDER THE...|  5652|
|            WARRANTS| 99821|
|               ARSON|  2633|
|FORGERY/COUNTERFE...| 22995|
|            GAMBLING|   105|
|             BRIBERY|   796|
|             ASSAULT|165762|
|         DRUNKENNESS|  9760|
|           EXTORTION|   339|
|                TREA|    14|
|         WEAPON LAWS| 21004|
|           LOITERING|  1639|
|      SUSPICIOUS OCC| 79087|
|             ROBBERY| 54467|
+--------------------+------+
only showing top 20 rows



### Incidents anlyzed by day of the week

In [12]:
df2.groupBy('DayOfWeek').count().show()

+---------+------+
|DayOfWeek| count|
+---------+------+
|Wednesday|310982|
|  Tuesday|301853|
|   Friday|323562|
| Thursday|303303|
| Saturday|308380|
|   Monday|294007|
|   Sunday|283464|
+---------+------+



### Crime in different districts of SF

In [13]:
df2.filter("PdDistrict !='NA'").groupBy('PdDistrict').count().show()

+----------+------+
|PdDistrict| count|
+----------+------+
|   MISSION|288278|
|   BAYVIEW|205078|
|   CENTRAL|221433|
|   TARAVAL|155174|
|TENDERLOIN|186637|
| INGLESIDE|180819|
|      PARK|119532|
|  SOUTHERN|390118|
|  RICHMOND|112620|
|  NORTHERN|265861|
+----------+------+



In [14]:
dft=df2.withColumn('Date', to_date(col('Date'), "M/d/y"))\
.withColumn('Month', month(col('Date'))).withColumn('Year', year(col('Date')))

dfh = dft.withColumn('Hour', hour(dft.Time))

dfh.show(5)


+--------------+-------------+-------------+--------------------+---------+----------+-------------------+----------+-----------------+----------------+-----+----+----+
|          PdId|Incident Code|     Category|            Descript|DayOfWeek|      Date|               Time|PdDistrict|                X|               Y|Month|Year|Hour|
+--------------+-------------+-------------+--------------------+---------+----------+-------------------+----------+-----------------+----------------+-----+----+----+
| 4133422003074|         3074|      ROBBERY|ROBBERY, BODILY F...|   Monday|2004-11-22|2023-05-16 17:50:00| INGLESIDE|-122.420084075249|37.7083109744362|   11|2004|  17|
| 5118535807021|         7021|VEHICLE THEFT|   STOLEN AUTOMOBILE|  Tuesday|2005-10-18|2023-05-16 20:00:00|      PARK|           -120.5|            90.0|   10|2005|  20|
| 4018830907021|         7021|VEHICLE THEFT|   STOLEN AUTOMOBILE|   Sunday|2004-02-15|2023-05-16 02:00:00|  SOUTHERN|           -120.5|            90.0|   

### Incidents anlyzed by different time components

In [15]:
dfh.groupBy('Hour').count().orderBy((col('count')).desc()).show()

+----+------+
|Hour| count|
+----+------+
|  18|135559|
|  17|129958|
|  12|127336|
|  19|121732|
|  16|120152|
|  15|114376|
|  20|110489|
|  22|109828|
|   0|108669|
|  14|107298|
|  21|105353|
|  13|103913|
|  23|101240|
|  11| 93405|
|  10| 91483|
|   9| 85510|
|   8| 78203|
|   1| 63169|
|   2| 52750|
|   7| 52242|
+----+------+
only showing top 20 rows



In [16]:
dfh.groupBy('Year').count().orderBy((col('count')).desc()).show()

+----+------+
|Year| count|
+----+------+
|2015|151164|
|2017|149204|
|2013|147436|
|2016|145735|
|2014|144631|
|2003|142531|
|2004|141723|
|2005|136675|
|2012|135271|
|2008|134986|
|2009|134052|
|2006|131619|
|2007|131451|
|2010|127563|
|2011|126537|
|2018| 44973|
+----+------+



In [17]:
dfh.groupBy('Month').count().orderBy((col('count')).desc()).show()

+-----+------+
|Month| count|
+-----+------+
|    1|189311|
|    3|189056|
|   10|182706|
|    4|181929|
|    8|180865|
|    5|179593|
|    9|176098|
|    7|175177|
|    2|170790|
|   11|167901|
|    6|167877|
|   12|164248|
+-----+------+



### correlation analysis

In [18]:
#df_all=dfh.groupBy('PdDistrict','Category','Descript','DayOfWeek','Month','Year','Hour').count().orderBy((col('count')).desc())
p=dfh.select('Year','Month','Hour','X','Y','Incident Code')

numerical=[ 'Year','Month','Hour','X','Y','Incident Code']
# create a correlations matrix:
n_numerical = len(numerical)
corr_pair = []
for i in range(0, n_numerical):
    temp = [] * i
    for j in range(i+1, n_numerical):
        temp.append((numerical[i], numerical[j],p.corr(numerical[i], numerical[j])))
    corr_pair.append(temp)
    
display(sc.parallelize(corr_pair).flatMap(lambda x:x).collect())



[('Year', 'Month', -0.03621290790179111),
 ('Year', 'Hour', 0.00012307117358386268),
 ('Year', 'X', -0.012353832737066773),
 ('Year', 'Y', -0.008930261980690002),
 ('Year', 'Incident Code', 0.007481539234277595),
 ('Month', 'Hour', 0.001569769800269952),
 ('Month', 'X', 0.0035197335011096474),
 ('Month', 'Y', 0.0023665843855859192),
 ('Month', 'Incident Code', -0.009223815709996482),
 ('Hour', 'X', -0.00020092057827674408),
 ('Hour', 'Y', -0.0005525643676066241),
 ('Hour', 'Incident Code', -0.037512693512052664),
 ('X', 'Y', 0.5324418445774036),
 ('X', 'Incident Code', 0.0024046637544743875),
 ('Y', 'Incident Code', -0.0016844887746580576)]

In [19]:
d1=dfh.filter("PdDistrict !='NA'").select(col('Category'),col('Descript'))

df_data=d1.withColumn('Category_1',lower(col('Category')))\
.withColumn('Description',lower(col('Descript'))).drop('Category','Descript')\
.withColumnRenamed('Category_1', 'Category')

df_data=df_data.select(str.strip('Description'),str.strip('Category'))
                 

### Data ready for classification

In [20]:
df_data.show()

+--------------------+--------------+
|         Description|      Category|
+--------------------+--------------+
|robbery, bodily f...|       robbery|
|   stolen automobile| vehicle theft|
|   stolen automobile| vehicle theft|
|             battery|       assault|
|             battery|       assault|
|             battery|       assault|
|stolen and recove...| vehicle theft|
|             battery|       assault|
|         trespassing|      trespass|
|burglary of resid...|      burglary|
|grand theft from ...| larceny/theft|
|enroute to depart...|      warrants|
|drivers license, ...|other offenses|
|drivers license, ...|other offenses|
|         trespassing|      trespass|
|petty theft shopl...| larceny/theft|
|robbery of a comm...|       robbery|
|possession of heroin| drug/narcotic|
|grand theft of pr...| larceny/theft|
|suspicious occurr...|suspicious occ|
+--------------------+--------------+
only showing top 20 rows

